In [275]:
library(dplyr)
library(ggplot2)
library(GGally)
library(reshape2)
library(gridExtra)


Attaching package: ‘gridExtra’

The following object is masked from ‘package:dplyr’:

    combine



In [408]:
#for simplicity, have one hyperparameters object that contains all variables
get_default_hyperparameters <- function(){
    
    hyperparameters <- list()
    
    #dataset-specific settings:
    attr(hyperparameters, "main_url") <- "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/"
    attr(hyperparameters, "files")    <- c("winequality-red.csv", "winequality-white.csv")
    attr(hyperparameters, "response_name") <- "quality"
    
    
    attr(hyperparameters, "ensemble_K")  <- 5
    #number of folds of cross-validation for evaluating entire ensembles
    attr(hyperparameters, "top_level_repeats") <- 10
    #number of repeats to perform at top level (evaluating entire ensemble-forming strategy)
    attr(hyperparameters, "ensemble_size") <- 3
    attr(hyperparameters, "n_attempts") <- 10
    attr(hyperparameters, "single_model_K") <- 5
    #number of folds of cross-validation for finding single models
    attr(hyperparameters, "starting_power") <- 1
    attr(hyperparameters, "max_power") <- 4
    attr(hyperparameters, "weight_by_scores") <- TRUE
    attr(hyperparameters, "run_standard_model") <- FALSE
    attr(hyperparameters, "new_features") <- FALSE
    return(hyperparameters)
}

In [120]:
#this function does all preprocessing (normalization etc.)
#and defines new features by applying transformations
preprocess <- function(data, response_name, hyperparameters=get_default_hyperparameters()){
    
    #separate into response and predictors
    response <- data[[response_name]]
    predictors <- data
    predictors[[response_name]] <- NULL
        
    #z-score predictors
    znorm <- function(x)  (x - mean(x))/ sd(x)
    predictors <- apply(predictors, 2, znorm)
    
    predictors <- as.data.frame(predictors)
    
    #define new features
    if(attr(hyperparameters, "new_features")){
        original_features <- colnames(predictors)
        for(i in 1:length(original_features)){
            f <- original_features[[i]]
            predictors[[paste("log.",f,sep='')]] <- log(predictors[[f]] - min(predictors[[f]]) + 1)
            predictors[[paste("sqrt.",f,sep='')]] <- sqrt(predictors[[f]] - min(predictors[[f]]))
            predictors[[paste("inv.",f,sep='')]] <- 1/(predictors[[f]] - min(predictors[[f]] + 1))
        }
    }
    
    
    
    #combine into single matrix again
    data <- cbind(response, predictors)
    colnames(data)[1] <- "y"
    
    return(data)
    
}

In [453]:
run_ensemble <- function(ensemble_models, data, train_indices, test_indices, scores=NULL, hyperparameters=get_default_hyperparameters(), verbose=FALSE){
    
    weight_by_scores <- attr(hyperparameters, "weight_by_scores")
    response_name <- attr(hyperparameters, "response_name")
    
    ensemble_size <- length(ensemble_models)
    predictions <- vector("list", ensemble_size)
    for (n in 1:ensemble_size){
        if(verbose){
            print(paste("running model", n, "/", ensemble_size, sep=" "))
        }
        
        powers <- ensemble_models[[n]]
        
        predictions[[n]] <- polyreg(data, powers, train_indices, test_indices)
    }
    predictions_matrix <- t(Reduce(rbind, predictions))
    
    weights = rep(1/length(ensemble_models), length(ensemble_models))
    if(length(scores)>0 && weight_by_scores){
        weights = -log(scores)
        weights = weights/sum(weights)
    }
    else{
        #assume NULL was passed for scores
        weights = rep(1/length(ensemble_models), length(ensemble_models))
    }
    weighted_avg <- rep(0, nrow(data))
    for (n in 1:ensemble_size){    
        weighted_avg <- weighted_avg + weights[n]*predictions_matrix[,n]
    }
    
    return(weighted_avg)
}

In [399]:
find_ensemble <- function(data, hyperparameters=get_default_hyperparameters(), verbose=FALSE){
    
    
    ensemble_size <- attr(hyperparameters, "ensemble_size")
    ensemble = vector("list", ensemble_size)
    scores = rep(0, ensemble_size)
    
    for (n in 1:ensemble_size){
    
        if(verbose){
            print(paste("finding model", n, "/", ensemble_size, sep=" "))
        }
        outputs <- find_single_model(data, hyperparameters=hyperparameters, verbose=verbose)
        scores[n] <- outputs[[1]]
        ensemble[[n]] <- outputs[[2]]
    }
    return(list(scores, ensemble))
}

In [464]:
#function for polynomial-regression-based prediction
polyreg <- function(data, powers, train_indices, test_indices){
    
    response <- data$y
    predictors <- data
    predictors[["y"]] <- NULL
    pred_poly_matrix <- NULL
    for(p in 1:ncol(predictors)){
        if(powers[[p]] > 0){
            m <- poly(predictors[[p]], powers[[p]])
            colnames(m) <- paste(colnames(predictors)[p], colnames(m), sep="")
            if(is.null(pred_poly_matrix)){
                pred_poly_matrix <- m
            }
            else{
                pred_poly_matrix <- cbind(pred_poly_matrix, m)
            }
        }
    }
    full_data <- as.data.frame(cbind(response, pred_poly_matrix))
    colnames(full_data)[1] <- "y"

    print("until reg")
    
    model  <- lm(y ~ (.)^2, data=full_data[train_indices,])
            
    return(predict(model, newdata=full_data[test_indices,]))
}

In [454]:
#find a single model
#to serve as part of an ensemble
#return set of predictors used for that model
find_single_model <- function(data, hyperparameters=get_default_hyperparameters(), verbose=FALSE){
    
    K <- attr(hyperparameters, "single_model_K")
    CV_folds   <- get_CV_folds(nrow(data), K)
    train_indices <- CV_folds[[1]]
    test_indices <- CV_folds[[2]]
    n_attempts <- attr(hyperparameters, "n_attempts")
    max_power  <- attr(hyperparameters, "max_power")
    starting_power <- attr(hyperparameters, "starting_power")
    
    response <- data$y
    predictors <- data
    predictors[["y"]] <- NULL
    n_predictors <- ncol(predictors)
    
    current_powers <- rep(starting_power, n_predictors)
    best_powers <- rep(starting_power, n_predictors)
    is_dummy <- rep(FALSE, n_predictors)
    for(i in 1:length(is_dummy)){
        is_dummy[i] <- length(unique(predictors[[i]])) <= 2
    }
    
    best_score <- Inf
    
    null_model_score <- 0
    
    for(n in 1:n_attempts){
        
        if(n == 1){
            #do nothing here in the first iteration
        }
        else{
            #in iterations after the first, make random changes to the model
            
            
            
            index <- sample(1:n_predictors, 1)
            sign  <- sample(c(-1, 1), 1)
            if(current_powers[index] == max_power && sign==1){
                sign <- -1
            }
            if(current_powers[index]==0 && sign==-1){
                sign <-  1
            }
            if(is_dummy[index] && current_powers[index] == 1){
                sign <- -1
            }
            
            current_powers[index] <- current_powers[index] + sign
            
            
            
        }
        if(verbose){
            print(paste("Evaluating model:", paste(current_powers, sep=" ", collapse=", "), sep=" "))
        }
        
        oof_scores <- rep(0, K)
        
        
        
        for(k in 1:K){
            if(sum(current_powers)==0){
                #empty model - just use mean as intercept
                train_Y <- response[train_indices[[k]]]
                test_Y  <- response[test_indices[[k]]] 
                prediction <- mean(train_Y)
                oof_scores[k] <- mean(abs(test_Y - prediction))
            }
            else{
                
                
                    
                y_pred <- polyreg(data, current_powers, train_indices[[k]], test_indices[[k]])
            
                oof_scores[k] <- mean(abs(response[test_indices[[k]]] - y_pred))
            }

        }#end loop over folds
        
        if(n==1){
            starting_model_score <- mean(oof_scores)
        }
        
        #if the model is an improvement over previous best, update it
        final_score <- mean(oof_scores)
        
        if(final_score < best_score){
            output_message = "Keeping model"
            best_score <- final_score
            best_powers <- current_powers
        }
        else{
            output_message = "Discarding model"
            current_powers <- best_powers
        }
        if(verbose){
            print(paste("Model score =", final_score, sep=" "))
            print(output_message)
        }
    }
    
    
    #return description of this model
    #as a list of predictor names (strings)
    return(list(best_score/starting_model_score, best_powers))
}

In [402]:
#check if data files are present in local directory
#if they are not, try to download
#return 1 if files are now present locally (either found or successfully downloaded)
#return 0 otherwise
#if this returns 1, we expect a subsequent call to read_data to succeed
find_or_download_data <- function (main_url, files){

    for (i in 1:length(files)){
        if(!file.exists(files[i])){
            print(paste("Could not find", files[i], "- attempting to download", sep=" " ))
            download.file(paste(main_url, files[i],sep="") , files[i], "auto", quiet = FALSE)
        }
    }
    for (i in 1:length(files)){
        if(!file.exists(files[i])){
            print(paste("Could not find or download", files[i], "- training will fail", sep=" " ))
            return(0)
        }
    }
    return(1)
}

In [403]:
#read data from .csv files to data.frame
#merge multiple .csv into one (binds rows)
#add a one-hot encoding of the file number
read_data <- function(files){
    single_datasets <- vector("list", length(files))
    for (i in 1:length(files)){
        single_datasets[[i]] <- read.csv(files[i], header=TRUE, sep=";")
        single_datasets[[i]]$file.number <- i
    }
    data <- bind_rows(single_datasets) 
    #bind_rows : dplyr function
    
    #data is now full dataset (merged across all files)
        
    #make one-hot encoding of file number
    u <- unique(data$file.number)
    for (val in u[1:(length(u)-1)]){
        data[paste("file.number.",val,sep="")] <- (data$file.number==val)*1.0
    }
    data$file.number <- NULL
    #remove initial (categorical) file number column
    return(data)
}

In [404]:
get_CV_folds <- function(N, K){
    train_indices <- vector("list", K)
    test_indices  <- vector("list", K)
    random_permutation <- sample(N, N, replace=FALSE)
    for (i in 1:K){
        start <- floor(N*(i-1)/K) + 1
        stop  <- floor(N*i/K)
        test_indices[[i]] <-  random_permutation[start:stop]
        train_indices[[i]] <- random_permutation[-(start:stop)]
    }
    return(list(train_indices, test_indices))
}

In [405]:
#define shorter names for this specific dataset
#to help plotting
short_names <- function(){
    sn <- list(fixed.acidity="fix.acd",
               volatile.acidity="vol.acd",
                citric.acid="cit.acd",
                residual.sugar="res.sug",
                chlorides="chlor",
                free.sulfur.dioxide="fr.SO2",
                total.sulfur.dioxide="tot.SO2",
                density="dens",
                pH="pH",
                sulphates="sulph",
                alcohol="alc",
                file.number.1="red")
    return(sn)
}

In [460]:
#main function for evaluating different approaches
evaluate_ensemble <- function(hyperparameters){

    verbose <- attr(hyperparameters, "verbose")
    K <- attr(hyperparameters, "ensemble_K")
    R <- attr(hyperparameters, "top_level_repeats")
    
    main_url <- attr(hyperparameters, "main_url")
    files    <- attr(hyperparameters, "files")
    response_name <- attr(hyperparameters, "response_name")
    
    have_data <- find_or_download_data(main_url, files)
    if(!have_data){
        print("Could not find or download data - returning null")
        return(NULL)
    }
    else{
        
        data <- read_data(files)
        
        data <- preprocess(data, response_name)
        
        CV_indices <- get_CV_folds(nrow(data), K)
        train_indices <- CV_indices[[1]]
        test_indices  <- CV_indices[[2]]
        
        score_matrix = matrix(0, R, K)
        
        for (r in 1:R){
            if(verbose){
                #print(paste('Top-level repeat', r, sep=' '))
            }
            for (k in 1:K){
                if(verbose){
                    #print(paste('Top-level CV fold', k, sep=' '))
                }
                if(attr(hyperparameters, "run_standard_model")){
                    #just fit one linear model to original predictors
                    #for comparison
                    train_data <- data[train_indices[[k]],]
                    test_data  <- data[test_indices[[k]],]
                    model  <- lm(y ~ (.)^2, data=train_data)
                    score_matrix[r,k]  <- mean(abs(test_data$y- predict(model, newdata=test_data)))
                }
                else{
                    #first step: find an ensemble of models
                    #by searching the space of possible models
                    #(this is the space of 2^P possible subsets of all predictors)
                    outputs <- find_ensemble(data[train_indices[[k]],])
                    scores <- outputs[[1]]
                    ensemble_definitions <- outputs[[2]]
                    if(verbose){
                        print('Model definitions:')
                        print(ensemble_definitions)
                        print('Scores:')
                        print(scores)
                    }
                    #now train the ensemble on the same data
                    ensemble_models <- train_ensemble(ensemble_definitions, data[train_indices[[k]],])
                    
                    predictions <- run_ensemble(ensemble_models, data, train_indices[[k]], test_indices[[k]], scores, hyperparameters)
                    score_matrix[r,k] <- mean(abs(predictions - data$y[test_indices[[k]]]))
                    #print(paste("Ensemble score =",score,sep=" "))
                }
            }
        }
    }
    dim(scores) <- NULL
    return(scores)
}

In [465]:
#top level code: define hyperparameters, run ensemble
hp <- get_default_hyperparameters()

attr(hp, "verbose") <- TRUE

attr(hp, "run_standard_model") <- TRUE
scores_1 <- evaluate_ensemble(hp)
attr(hp, "run_standard_model") <- FALSE
scores_2 <- evaluate_ensemble(hp)

print(paste("mean of scores_1 = ", mean(scores_1), sep=" "))
print(paste("mean of scores_2 = ", mean(scores_2), sep=" "))
print(paste("SEM of scores_1 = ", sd(scores_1)/sqrt(length(scores_1)), sep=" "))
print(paste("SEM of scores_2 = ", sd(scores_2)/sqrt(length(scores_2)), sep=" "))

[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "unt

Warning message in predict.lm(model, newdata = full_data[test_indices, ]):
“prediction from a rank-deficient fit may be misleading”

[1] "until reg"


Warning message in predict.lm(model, newdata = full_data[test_indices, ]):
“prediction from a rank-deficient fit may be misleading”

[1] "until reg"


...

[1] "until reg"


Warning message in predict.lm(model, newdata = full_data[test_indices, ]):
“prediction from a rank-deficient fit may be misleading”

[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "until reg"
[1] "Model definitions:"
[[1]]
 [1] 1 1 1 1 0 2 2 1 1 1 1 1

[[2]]
 [1] 1 1 1 0 0 2 2 1 1 1 1 1

[[3]]
 [1] 1 2 1 1 1 1 1 1 1 2 1 1

[1] "Scores:"
[1] 0.9901498 0.9909394 0.9879349


Warning message in model.matrix.default(mt, mf, contrasts):
“the response appeared on the right-hand side and was dropped”Warning message in model.matrix.default(mt, mf, contrasts):
“problem with term 1 in model.matrix: no columns are assigned”Warning message in model.matrix.default(mt, mf, contrasts):
“the response appeared on the right-hand side and was dropped”Warning message in model.matrix.default(mt, mf, contrasts):
“problem with term 1 in model.matrix: no columns are assigned”Warning message in model.matrix.default(mt, mf, contrasts):
“the response appeared on the right-hand side and was dropped”Warning message in model.matrix.default(mt, mf, contrasts):
“problem with term 1 in model.matrix: no columns are assigned”Warning message in if (powers[[p]] > 0) {:
“the condition has length > 1 and only the first element will be used”Warning message in if (powers[[p]] > 0) {:
“the condition has length > 1 and only the first element will be used”

ERROR: Error in polym(x, ..., degree = degree, coefs = coefs, raw = raw): arguments must have the same length


In [385]:
#main function for EDA
library(gridExtra)
library(ggplot2)
library(reshape2)


reorder_cormat <- function(cormat){
    # Use correlation between variables as distance
    dd <- as.dist((1-cormat)/2)
    hc <- hclust(dd)
    cormat <-cormat[hc$order, hc$order]
}

get_breaks <- function(x, margin, n_breaks){
    m <- margin*(max(x) - min(x))
    
    return(seq(min(x) + m, max(x) - m, (max(x) - min(x) - 2*m)/(n_breaks-1)))
}

run_eda <- function(to_show, hyperparameters, corr_cutoff=0.5, boxplot_predictor_groups=3){
    main_url <- attr(hyperparameters, "main_url")
    files    <- attr(hyperparameters, "files")
    response_name <- attr(hyperparameters, "response_name")
    
    have_data <- find_or_download_data(main_url, files)
    if(!have_data){
        print("Could not find or download data - returning null")
        return(NULL)
    }
    else{
        data <- read_data(files)
        
        #plot relationship between each predictor and response
        predictor_names <- colnames(data)[colnames(data) != response_name]
        
        if(to_show=="pairs"){
        #pairs plot
            
            sn <- short_names()
            labels <- list()
            for (i in 1:length(predictor_names)){
                labels[i] <- sn[[predictor_names[i]]]
            }
        
            pairs(data[,predictor_names], labels=labels, panel=points, pch = 16, cex = .5, xaxt = "n", yaxt = "n")
            
            #alternative: ggally pairs plot
            #this does not handle the large number of variables well
        
            ## code for ggally pairs plot:
            #pairplot_columns <- 1:length(colnames(data))
            #pairplot_columns <- pairplot_columns[colnames(data) != response_name]
            #pairplot <- ggpairs(data, columns=pairplot_columns) + 
            #  ggtitle("Pairs plot")
            #print(pairplot)
        }
        else if(to_show=="corr_heatmap"){
        #correlation heatmap
        
            predictors <- data[,predictor_names]
            colnames(predictors) <- labels
        
            cormat <- round(cor(predictors),2)
            cormat <- reorder_cormat(cormat)
        
            cormat[lower.tri(cormat)]<- NA
        
            # Melt the correlation matrix
            melted_cormat <- melt(cormat, na.rm = TRUE)
            # Heatmap
            ggplot(data = melted_cormat, aes(Var2, Var1, fill = value))+
                 geom_tile(color = "white")+
                 scale_fill_gradient2(low = "blue", high = "red", mid = "white", 
                 midpoint = 0, limit = c(-1,1), space = "Lab", 
                 name="Pearson\nCorrelation") +
                 theme_minimal()+ 
                 theme(axis.text.x = element_text(angle = 45, vjust = 1, 
                 size = 12, hjust = 1))+
                 coord_fixed()
        }
        else if(to_show=="corr_values"){
        #print pairs of variables with largest absolute correlation values

            melted_cormat <- melt(cormat, na.rm=TRUE)
            melted_cormat <- melted_cormat[order(abs(melted_cormat$value)),]
            print(melted_cormat[abs(melted_cormat$value) > corr_cutoff & melted_cormat$Var1 != melted_cormat$Var2,])
        }
        else if(to_show=="response_hist"){
        #plot histogram of response values
            par(pin=c(4,3))
            vals <- data[[response_name]]
            h <- hist(vals, xlab=response_name, ylab="count", breaks=seq(min(vals)-0.5, max(vals) + 0.5, 1), 
                 main=paste("Histogram of ", response_name, sep=" "),ylim=c(0, 3000))
            #plot(h, )
            scale <- length(vals)
            curve(dnorm(x, mean=mean(vals), sd=sd(vals))*scale, add=TRUE, col="darkblue", lwd=2)

        }
        else if(to_show=="boxplots_grouped_by_response"){
        #plot series of boxplots grouped by values of response (on x axis)
        #only works if response takes discrete values
        #has some limitations, but can be useful
            
            label_format <- function(x) sprintf("%.2f", x)
            breaks <- function(x) get_breaks(x, 0.05, 4)
            
            nplot <- length(predictor_names)
            nrow <- floor(sqrt(nplot))
            ncol <- ceiling(nplot/nrow)
            plots <- list()
            for(i in 1:length(predictor_names)){
                x <- predictor_names[i]
                if(length(unique(data[[x]])) > 2){
                    #exclude any one-hot encodings
                    plots[[i]] <- ggplot(data, aes_string(group=response_name, y = x, x = response_name)) + 
                        geom_boxplot(outlier.size=0.15) + geom_smooth(aes(group=1), method="lm") + 
                        scale_y_continuous(labels=label_format) + 
                        theme(axis.text.x = element_text(color = "grey20", size = 8, angle = 90, hjust = .5, vjust = .5, face = "plain")) +
                        coord_flip()
                    #have to add aes(group=1) to make smoothing work
                }
            }
            grid.arrange(grobs=plots, ncol=ncol)
        }
        else if(to_show=="boxplots_predictor_on_x"){
        #plot series of boxplots grouped by values of predictor (on x axis)  
        #divide predictor into quantiles for grouping
        #this tends to give ugly plots with issues that make them hard to interpret
        #not used further
            qstep <- 1/boxplot_predictor_groups
            nplot <- length(predictor_names)
            nrow <- floor(sqrt(nplot))
            ncol <- ceiling(nplot/nrow)
            plots <- list()
            for(i in 1:length(predictor_names)){
                x <- predictor_names[i]
                if(length(unique(data[[x]])) > 2){
                    #exclude any one-hot encodings
                    quantile_boundaries <- quantile(data[[x]], probs=seq(0,1,qstep))
                    quantile_groups <- rep(0, length(data[[x]]))
                    for (q in quantile_boundaries){
                        quantile_groups <- quantile_groups + 1.0*(data[[x]] > q)
                    }
                    plots[[i]] <- ggplot(data, aes_string(group=quantile_groups, y = response_name, x = x)) + 
                    geom_boxplot(outlier.size=0.15) + geom_smooth(aes(group=1), method="lm") 
                        
                    #have to add aes(group=1) to make smoothing work
                }
            }
            grid.arrange(grobs=plots, ncol=ncol)
        }
        else if(to_show=="separate_regressions"){
            
            label_format <- function(x) sprintf("%.2f", x)
            
            nplot <- length(predictor_names)
            nrow <- floor(sqrt(nplot))
            ncol <- ceiling(nplot/nrow)
            plots <- list()
            for(i in 1:length(predictor_names)){
                x <- predictor_names[i]
                if(length(unique(data[[x]])) > 2){
                    #exclude any one-hot encodings
                    plots[[i]] <- ggplot(data, aes_string(y = x, x = response_name)) + 
                        geom_count() + geom_smooth(aes(group=1), method="lm") + theme(legend.position = "none") +
                        scale_y_continuous(labels=label_format) + 
                        theme(axis.text.x = element_text(color = "grey20", size = 8, angle = 90, hjust = .5, vjust = .5, face = "plain")) +
                        coord_flip()
                    #have to add aes(group=1) to make smoothing work
                }
            }
            grid.arrange(grobs=plots, ncol=ncol)
        } 
    }
}